# AL Results Visualization

In this notebook, the goal is to visualize the results and compare the performance of the individual strategies with the respective model classes.

# Setup

In [ ]:
# define the paths
import sys

basepath = "../"  # Project directory
sys.path.append(basepath)

# AL Scripts
AL_SCRIPTS_PATH = basepath + "al_lib/"

sys.path.append({AL_SCRIPTS_PATH})

from al_lib.active_learning_setting import FeatureImportancePaths

PATHS = FeatureImportancePaths()
(DATA_PATH, FIGURE_PATH, ENV_PATH, RESULTS_PATH, LOG_DIR) = PATHS
# sys.path.extend(PATHS)

sys.path

['/home/fhwn.ac.at/202375/.conda/envs/thesis/lib/python312.zip',
 '/home/fhwn.ac.at/202375/.conda/envs/thesis/lib/python3.12',
 '/home/fhwn.ac.at/202375/.conda/envs/thesis/lib/python3.12/lib-dynload',
 '',
 '/home/fhwn.ac.at/202375/.conda/envs/thesis/lib/python3.12/site-packages',
 '../',
 {'../al_lib/'},
 '../',
 '../04_Active_Learning/results/figures/',
 '/home/fhwn.ac.at/202375/.conda/envs/thesis/lib',
 '../data/',
 '../04_Active_Learning/results/',
 '../',
 {'../al_lib/'},
 '../',
 '../04_Active_Learning/results/figures/',
 '/home/fhwn.ac.at/202375/.conda/envs/thesis/lib',
 '../data/',
 '../04_Active_Learning/results/',
 '../',
 {'../al_lib/'},
 '../',
 '../04_Active_Learning/results/figures/',
 '/home/fhwn.ac.at/202375/.conda/envs/thesis/lib',
 '../data/',
 '../04_Active_Learning/results/',
 '../',
 {'../al_lib/'},
 '../',
 '../04_Active_Learning/results/figures/',
 '/home/fhwn.ac.at/202375/.conda/envs/thesis/lib',
 '../data/',
 '../04_Active_Learning/results/',
 '../',
 {'../al_l